In [24]:
import pandas as pd
from faker import Faker
import random
from functools import reduce

In [25]:
def format_harga(harga):
    return f"IDR {harga:,.0f}".replace(",", ".")

In [26]:
def generate_data(num_days=4, products_per_day=3):
    faker = Faker()
    dates = pd.date_range("2024-08-06", periods=num_days).strftime('%Y-%m-%d')

    food_names = [
        f"{faker.word()} {random.choice(['Pizza', 'Burger', 'Pasta', 'Salad', 'Sushi', 'Soup', 'Steak', 'Sandwich'])}"
        for _ in range(products_per_day * num_days)
    ]

    data_penjualan = pd.DataFrame({
        "id": ["GN" + str(random.randint(100, 999)) for _ in range(products_per_day * num_days)],
        "nama": food_names,
        "harga": [random.randint(20000, 200000) for _ in range(products_per_day * num_days)],
        "quantity": [random.randint(1, 100) for _ in range(products_per_day * num_days)],
        "tanggal": [date for date in dates for _ in range(products_per_day)]
    })

    data_penjualan['harga_formatted'] = list(map(format_harga, data_penjualan['harga']))
    return data_penjualan

data_penjualan = generate_data()
print(data_penjualan[['id', 'nama', 'harga_formatted', 'quantity', 'tanggal']])

       id             nama harga_formatted  quantity     tanggal
0   GN694     field Burger     IDR 110.446        76  2024-08-06
1   GN965  physical Burger     IDR 165.921        44  2024-08-06
2   GN739     others Sushi      IDR 78.465        34  2024-08-06
3   GN638     total Burger      IDR 73.477        92  2024-08-07
4   GN568  finish Sandwich      IDR 75.371        78  2024-08-07
5   GN370          a Pizza     IDR 107.402        24  2024-08-07
6   GN340   marriage Pizza     IDR 185.979        17  2024-08-08
7   GN305  something Pizza      IDR 55.620        78  2024-08-08
8   GN662       side Pizza     IDR 123.240        83  2024-08-08
9   GN204         out Soup     IDR 189.297        11  2024-08-09
10  GN485        bad Pizza      IDR 83.032        56  2024-08-09
11  GN983        nor Salad      IDR 28.516         3  2024-08-09


In [27]:
def hitung_pendapatan(df):
    df['pendapatan'] = df['harga'] * df['quantity']
    df['pendapatan_formatted'] = list(map(format_harga, df['pendapatan']))
    return df[['id', 'nama', 'harga_formatted', 'quantity', 'tanggal', 'pendapatan', 'pendapatan_formatted']]

data_with_pendapatan = hitung_pendapatan(data_penjualan)
print(data_with_pendapatan.to_string())

       id             nama harga_formatted  quantity     tanggal  pendapatan pendapatan_formatted
0   GN694     field Burger     IDR 110.446        76  2024-08-06     8393896        IDR 8.393.896
1   GN965  physical Burger     IDR 165.921        44  2024-08-06     7300524        IDR 7.300.524
2   GN739     others Sushi      IDR 78.465        34  2024-08-06     2667810        IDR 2.667.810
3   GN638     total Burger      IDR 73.477        92  2024-08-07     6759884        IDR 6.759.884
4   GN568  finish Sandwich      IDR 75.371        78  2024-08-07     5878938        IDR 5.878.938
5   GN370          a Pizza     IDR 107.402        24  2024-08-07     2577648        IDR 2.577.648
6   GN340   marriage Pizza     IDR 185.979        17  2024-08-08     3161643        IDR 3.161.643
7   GN305  something Pizza      IDR 55.620        78  2024-08-08     4338360        IDR 4.338.360
8   GN662       side Pizza     IDR 123.240        83  2024-08-08    10228920       IDR 10.228.920
9   GN204         ou

In [28]:
def average_penjualan(df, tanggal, min_revenue=500000):
    try:
        # Filter rows based on date
        sales_on_date = df[df['tanggal'] == tanggal]
        if sales_on_date.empty:
            raise ValueError("Tanggal tidak ditemukan.")

        # Filter to include only rows where 'pendapatan' >= min_revenue
        high_revenue_sales = sales_on_date[sales_on_date['pendapatan'] >= min_revenue]
        
        # If no rows meet the condition, return a message
        if high_revenue_sales.empty:
            return f"Tidak ada penjualan dengan pendapatan di atas {format_harga(min_revenue)} pada tanggal tersebut."

        # Reduce to calculate total revenue by summing 'pendapatan' for filtered rows
        total_sales = reduce(int.__add__, (high_revenue_sales['pendapatan']))
        avg_sales = total_sales / len(high_revenue_sales)
        return format_harga(int(avg_sales))

    except ValueError as e:
        return str(e)
    except Exception as e:
        return "Format tanggal tidak sesuai."
    
tanggal_input = input("skibidi")
average = average_penjualan(data_penjualan, tanggal_input)
print(f"Rata-rata penjualan pada tanggal {tanggal_input}: {average}")

Rata-rata penjualan pada tanggal 2024-08-06: IDR 6.120.743


In [29]:
def total_penjualan_recursive(grouped_data, index=0, results=None):
    if results is None:
        results = []
    
    # Base case for recursion
    if index >= len(grouped_data):
        return results
    
    row = grouped_data.iloc[index]
    # Append result for the current date
    results.append({"tanggal": row['tanggal'], "total_penjualan": format_harga(int(row['pendapatan']))})
    
    # Recursive call with incremented index
    return total_penjualan_recursive(grouped_data, index + 1, results)

grouped = data_with_pendapatan.groupby('tanggal')['pendapatan'].sum().reset_index()
total_sales_by_date = total_penjualan_recursive(grouped)
for penjualan in total_sales_by_date:
    print(f"Tanggal: {penjualan['tanggal']}, Total Penjualan: {penjualan['total_penjualan']}")

Tanggal: 2024-08-06, Total Penjualan: IDR 18.362.230
Tanggal: 2024-08-07, Total Penjualan: IDR 15.216.470
Tanggal: 2024-08-08, Total Penjualan: IDR 17.728.923
Tanggal: 2024-08-09, Total Penjualan: IDR 6.817.607
